In [56]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
# from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd
from config import api_key

In [57]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.FollowTheMoney
members_collection = db.members
receipt_totals_collection = db.receiptTotals
candidate_data_collection = db.candidate_data
candidate_data_collection.remove()
members_collection.remove()
receipt_totals_collection.remove()

<ipython-input-57-ab10ed2b4962>:7: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  candidate_data_collection.remove()
<ipython-input-57-ab10ed2b4962>:8: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  members_collection.remove()
<ipython-input-57-ab10ed2b4962>:9: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  receipt_totals_collection.remove()


{'n': 627, 'ok': 1.0}

In [58]:
newurl=f"https://api.open.fec.gov/v1/schedules/schedule_a/by_state/by_candidate/?sort_null_only=false&api_key={api_key}&sort_hide_null=false&election_full=false&candidate_id=H8KS03155&candidate_id=H0KS02188&candidate_id=%20H8KS04112&candidate_id=H0KS01123&candidate_id=S0KS00315&candidate_id=S0KS00091&candidate_id=S8MO00160&candidate_id=S0MO00183&candidate_id=H8M001143&candidate_id=H2M002102&candidate_id=H8M009153&candidate_id=H0M004086&candidate_id=H4M005234&candidate_id=H0M006073&candidate_id=H0M007113&candidate_id=H4M008162&cycle=2014&cycle=2016&cycle=2018&cycle=2020&per_page=20&page=1&sort_nulls_last=false"
money=requests.get(newurl).json()

In [59]:
total_pages = money['pagination']['pages']
for i in range(1,total_pages+1):
    totals_url=f"https://api.open.fec.gov/v1/schedules/schedule_a/by_state/by_candidate/?sort_null_only=false&api_key={api_key}&sort_hide_null=false&election_full=false&candidate_id=H8KS03155&candidate_id=H0KS02188&candidate_id=20H8KS04112&candidate_id=H0KS01123&candidate_id=S0KS00315&candidate_id=S0KS00091&candidate_id=S8MO00160&candidate_id=S0MO00183&candidate_id=H8M001143&candidate_id=H2M002102&candidate_id=H8M009153&candidate_id=H0M004086&candidate_id=H4M005234&candidate_id=H0M006073&candidate_id=H0M007113&candidate_id=H4M008162&cycle=2014&cycle=2016&cycle=2018&cycle=2020&per_page=20&page={i}&sort_nulls_last=false"
    totals_json = requests.get(totals_url).json()
    for item in totals_json['results']:
        receipt_totals_collection.insert_one(item)

In [60]:
url=f"https://api.open.fec.gov/v1/candidates/?api_key={api_key}"
members = requests.get(f"{url}&election_year=2013&election_year=2014&election_year=2015&election_year=2016&election_year=2017&election_year=2018&election_year=2019&election_year=2020&sort=name&candidate_status=C&page=1&is_active_candidate=true&per_page=20&state=KS&state=MO&office=H&office=S&sort_hide_null=false&sort_null_only=false&sort_nulls_last=false").json()
#find out how many pages of data there are
total_pages = members['pagination']['pages']


In [61]:
#loop through and get all results
for i in range(1,total_pages+1):
    members = requests.get(f"{url}&election_year=2013&election_year=2014&election_year=2015&election_year=2016&election_year=2017&election_year=2018&election_year=2019&election_year=2020&sort=name&candidate_status=C&page={i}&is_active_candidate=true&per_page=20&state=KS&state=MO&office=H&office=S&sort_hide_null=false&sort_null_only=false&sort_nulls_last=false").json()
    #loop through each record and push to DB
    for item in members["results"]:
        members_collection.insert_one(item)

In [62]:
# grab the kansas canidates from tbe DB
KS_candidates = members_collection.find({"state":"KS"})
MO_candidates = members_collection.find({"state":"MO"})

In [63]:
#populat kansas
for cand in KS_candidates:
    candDetail_url = f"https://api.open.fec.gov/v1/candidate/{cand['candidate_id']}/filings/?sort=-receipt_date&sort_nulls_last=false&page=1&sort_hide_null=false&per_page=20&api_key=cKUzXnqasaxmD3l9mDOOD5wqRfZ0rnYuZJzxKnhx&sort_null_only=false"
    c_response = requests.get(candDetail_url).json()
    if c_response['results']:
        candidate_data_collection.insert_many(c_response['results'])

In [64]:
#populat MO
for cand in MO_candidates:
    candDetail_url = f"https://api.open.fec.gov/v1/candidate/{cand['candidate_id']}/filings/?sort=-receipt_date&sort_nulls_last=false&page=1&sort_hide_null=false&per_page=20&api_key=cKUzXnqasaxmD3l9mDOOD5wqRfZ0rnYuZJzxKnhx&sort_null_only=false"
    c_response = requests.get(candDetail_url).json()
    if c_response['results']:
        candidate_data_collection.insert_many(c_response['results'])

In [65]:
candidates = candidate_data_collection.find()
for candidate in candidates:
    print(candidate)

_funds': None, 'additional_bank_names': None, 'bank_depository_street_1': None, 'bank_depository_street_2': None, 'beginning_image_number': '202007249260815258', 'total_disbursements': None, 'fec_url': 'https://docquery.fec.gov/dcdev/posted/1428459.fec', 'is_amended': True, 'csv_url': 'https://docquery.fec.gov/csv/459/1428459.csv', 'debts_owed_by_committee': None, 'coverage_start_date': None, 'document_type': None}
{'_id': ObjectId('60a00d17b996a9aefc961cb9'), 'net_donations': None, 'cycle': 2020, 'update_date': '2019-02-08T00:00:00', 'document_description': 'Statement of candidacy 2019', 'pages': 2, 'total_communication_cost': None, 'pdf_url': 'https://docquery.fec.gov/pdf/212/201902089145478212/201902089145478212.pdf', 'bank_depository_zip': None, 'treasurer_name': 'WAGNER, ANN L.', 'committee_name': None, 'report_type': None, 'debts_owed_to_committee': None, 'report_year': 2019, 'form_type': 'F2', 'candidate_name': 'WAGNER, ANN L.', 'committee_type': None, 'total_receipts': None, 'f